<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/Longformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install accelerate
!pip -q install transformers datasets

import torch
torch.cuda.empty_cache()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import DatasetDict, Dataset, Features, Sequence, Value
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

# Step 1: Prepare DatasetDict with train, validation, and test splits
data = {
    "train": {
        "text": ["This is a training example.", "Another training sample here."],
        "labels": [[1, 0, 0], [0, 1, 1]],  # Multi-label (3 classes in this example)
    },
    "validation": {
        "text": ["This is a validation example.", "Another validation sample."],
        "labels": [[1, 0, 1], [0, 1, 0]],
    },
    "test": {
        "text": ["This is a test example.", "Another test sample."],
        "labels": [[1, 0, 0], [0, 1, 1]],
    },
}

# Convert to DatasetDict
dataset = DatasetDict({
    split: Dataset.from_dict(data_split, features=Features({
        'text': Value(dtype='string'),  # Keep the 'text' column
        'labels': Sequence(feature=Value(dtype='float32'))
    }))
    for split, data_split in data.items()
})

# Step 2: Load Longformer tokenizer
tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096")

# Tokenize datasets
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=1024, #2048, # 4096,  # Adjust max length for Longformer
        padding="max_length",
        truncation=True,
        #return_tensors="pt",
    )

def preprocess_labels(examples):
    """
    print(f"examples['labels']: {len(examples['labels'])} {examples['labels']}")
    print(f"examples['labels'][0]: {len(examples['labels'][0])} {examples['labels'][0]}")
    # Ensure labels are flattened and converted to float
    if isinstance(examples["labels"][0], list):  # Nested lists
        print("Nested list")
        flattened = [float(label) for sublist in examples["labels"] for label in sublist]
    else:  # Single list
        print("Single list")
        flattened = [float(label) for label in examples["labels"]]

    # Validate the length of labels
    #expected_length = 2  # Replace with the correct number of labels for your task
    #if len(flattened) != expected_length:
    #    raise ValueError(f"Labels length mismatch: Expected {expected_length}, got {len(flattened)}")

    examples["labels"] = flattened
    print(f"examples['labels']: {len(examples['labels'])} {examples['labels']}")
    return examples
    """
    # Convert each label list to a list of floats
    examples["labels"] = [[float(label_item) for label_item in label_list] for label_list in examples["labels"]]
    return examples


encoded_dataset = dataset.map(tokenize_function, batched=True)

print(f"encoded_dataset['train'][0]: {encoded_dataset['train'][0]}")  # Inspect a sample
#encoded_dataset = encoded_dataset.map(preprocess_labels, batched=True)
#print(f"encoded_dataset['train'][0]: {encoded_dataset['train'][0]}")  # Inspect a sample


# Step 3: Load Longformer model for sequence classification
model = LongformerForSequenceClassification.from_pretrained(
    "allenai/longformer-base-4096",
    num_labels=3,  # Number of labels in your multi-label problem
    problem_type="multi_label_classification",
)


# Step 4: Define Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = (logits > 0).astype(int)  # Convert logits to binary predictions
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="macro")
    roc_auc = roc_auc_score(labels, logits, average="macro", multi_class="ovr")
    return {"precision": precision, "recall": recall, "f1": f1, "roc_auc": roc_auc}

# Step 5: Set up Trainer
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # Reduce batch size
    per_device_eval_batch_size=1,  # Reduce batch size
    gradient_accumulation_steps=4,  # Simulate larger batch size
    fp16=True,  # Enable mixed precision
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

!nvidia-smi

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Step 6: Train the Model
trainer.train()




vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Initializing global attention on CLS token...


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Roc Auc
1,No log,0.693693,0.166667,0.333333,0.222222,0.666667
2,No log,0.693122,0.166667,0.333333,0.222222,0.666667
3,No log,0.692784,0.166667,0.333333,0.222222,0.666667


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=3, training_loss=0.3558661142985026, metrics={'train_runtime': 65.1146, 'train_samples_per_second': 0.092, 'train_steps_per_second': 0.046, 'total_flos': 3941138018304.0, 'train_loss': 0.3558661142985026, 'epoch': 3.0})

In [3]:
# Step 7: Evaluate the Model
results = trainer.evaluate(encoded_dataset["test"])
print("Test Results:", results)

Test Results: {'eval_loss': 0.6923288106918335, 'eval_precision': 0.16666666666666666, 'eval_recall': 0.3333333333333333, 'eval_f1': 0.2222222222222222, 'eval_roc_auc': 1.0, 'eval_runtime': 0.2165, 'eval_samples_per_second': 9.237, 'eval_steps_per_second': 9.237, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
